In [4]:
import datetime
import os
from stat import *
from shutil import copyfile
from config import Config
import pandas
import xlrd
import fnmatch

In [5]:
base = datetime.datetime.today()
date_list = [(base + datetime.timedelta(days=x)) for x in range(5)]

media_files=[]
try:
    for root, dirnames, filenames in os.walk(Config.MEDIA_FOLDER):
        for filename in fnmatch.filter(filenames, "*.mxf"):
            media_files.append(os.path.join(root, filename))
    print("Found {} mxf files in {}".format(len(media_files),Config.MEDIA_FOLDER))
except Exception as e:
    print('Failed to build file list on SAN drive: '+ str(e))
    raise e
    
cached_files=[]
try:
    for root, dirnames, filenames in os.walk(Config.CACHE_FOLDER):
        for filename in fnmatch.filter(filenames, "*.mxf"):
            cached_files.append(os.path.join(root, filename))
    print("Found {} mxf files in {}".format(len(cached_files),Config.CACHE_FOLDER))
except Exception as e:
    print('Failed to build file list on CACHE folder: '+ str(e))
    raise e

missing=set()

for lf in Config.LOCALIZED_FEEDS:
    for d in date_list:
        src_xls_filename=os.path.join(lf["xls"],d.strftime("%Y"),'Sent',d.strftime("%m")+"_"+d.strftime('%B'),d.strftime("%Y-%m-%d")+"_"+lf["log"]+".xls")
        if os.path.isfile(src_xls_filename):
            try:
                print("Searching in {}".format(src_xls_filename))
                df=pandas.read_excel(src_xls_filename)
                for c in df["Cart Number"].unique():
                    found_cart=False
                    for m in cached_files:
                        if c in m.decode('utf-8'):
                            # print("{} found in {}".format(c,m))
                            found_cart=True
                            break
                    if not found_cart:
                        # print("Adding {} to missing list.".format(c))
                        missing.add(c)
            except Exception as e:
                print('Failed to check missing media files: '+ str(e))
                continue
             
for m in missing:
    for f in media_files:
        if m in f.decode('utf-8'):
            print("Copy {} to {}".format(f,os.path.join(Config.CACHE_FOLDER,m+".mxf")))
            copyfile(f,os.path.join(Config.CACHE_FOLDER,m+".mxf"))
            break

                     

Found 16343 mxf files in /mnt/operations/Operations/
Found 512 mxf files in /ddrive/commercials
Searching in /mnt/fs_operations/04 Nickelodeon/Nickelodeon Hungary/Schedule/2019/Sent/09_September/2019-09-24_nickelodeon_HUN.xls
Searching in /mnt/fs_operations/04 Nickelodeon/Nickelodeon Hungary/Schedule/2019/Sent/09_September/2019-09-25_nickelodeon_HUN.xls
Searching in /mnt/fs_operations/04 Nickelodeon/Nickelodeon Romania/2019/Sent/09_September/2019-09-24_nickelodeonro.xls
Searching in /mnt/fs_operations/04 Nickelodeon/Nickelodeon Romania/2019/Sent/09_September/2019-09-25_nickelodeonro.xls
Searching in /mnt/fs_operations/04 Nickelodeon/Nickelodeon Czech/2019/Sent/09_September/2019-09-24_nickelodeoncze.xls
Searching in /mnt/fs_operations/04 Nickelodeon/Nickelodeon Czech/2019/Sent/09_September/2019-09-25_nickelodeoncze.xls
Searching in /mnt/fs_operations/04 Nickelodeon/Nickelodeon Bulgaria/2019/Sent/09_September/2019-09-24_nickelodeon_BUL.xls
Searching in /mnt/fs_operations/04 Nickelodeon/N